In [36]:
import geopandas
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from pyproj import CRS
from geopy.geocoders import Nominatim

In [37]:
pa_snow = pd.read_csv("snowstorm_PA.tsv", sep="\t")
pa_cities = pd.read_csv("PA_cities_counties.tsv", sep="\t")

In [38]:
pa_snow

,Location,Low End Snowfall,Expected Snowfall,High End Snowfall,"P>=0.1""","P>=1""","P>=2""","P>=4""","P>=6""","P>=8""","P>=12""","P>=18"""
0,Bethel Park,8,12,15,100%,100%,100%,98%,94%,83%,40%,1%
1,Franklin Park,8,13,16,100%,100%,100%,99%,96%,88%,51%,3%
2,Jefferson Hills,7,11,14,100%,100%,100%,98%,93%,80%,31%,1%
3,Monroeville,7,10,14,100%,100%,100%,99%,94%,81%,32%,1%
4,Moon/Pittsburgh Intl Airport,9,13,16,100%,100%,100%,99%,97%,91%,56%,4%
...,...,...,...,...,...,...,...,...,...,...,...,...
88,Ligonier,7,9,12,100%,100%,100%,100%,94%,71%,14%,0%
89,Lower Burrell,7,10,14,100%,100%,100%,99%,95%,82%,32%,1%
90,Monessen,7,10,13,100%,100%,100%,99%,93%,76%,26%,1%
91,Murrysville,7,10,14,100%,100%,100%,99%,96%,82%,31%,1%


In [39]:
pa_cities

,Name,Type,County,Class,Population (2018 Estimates),Incorporation date (as city),Sq Miles
0,Aliquippa,City,Beaver,Third,8908,1987,4.19
1,Allentown,City,Lehigh,Third,123828,1867,17.55
2,Altoona,City,Blair,Third,43702,1868,9.91
3,Arnold,City,Westmoreland,Third,4980,1939,0.73
4,Beaver Falls,City,Beaver,Third,8387,1928,2.13
5,Bethlehem,City,Lehigh + Northampton,Third,77283,1917,19.10
6,Bloomsburg,Town,Columbia,Third,14536,1929,4.35
7,Bradford,City,McKean,Third,8305,1879,3.35
8,Butler,City,Butler,Third,13107,1918,2.72
9,Carbondale,City,Lackawanna,Third,8447,1851,3.24


In [40]:
zip_path1 = "PaMunicipalities2025_01.zip"
zip_path2 = "tl_2021_us_county.zip"
!pip install pyogrio
us_county = gpd.read_file(f"zip://{zip_path1}", engine='pyogrio')
pa_municipalities = gpd.read_file(f"zip://{zip_path2}", engine='pyogrio')

In [41]:
us_county

,MSLINK,COUNTY,MUNICIPAL_,MUNICIPAL1,FIPS_MUN_C,FED_AID_UR,FIPS_COUNT,FIPS_AREA_,FIPS_AREA1,FIPS_SQ_MI,...,PSD_CODE_2,PSD_CODE_3,GIS_UPDATE,GIS_GEOMET,GPID,AREA,LEN,Shape_Leng,Shape_Area,geometry
0,1414,17,419,WESTOVER,83736,1,033,00000,None,2.6,...,None,None,0000/00/00,0000/00/00,70706,0.0,0.0,19731.121815,1.239526e+07,"POLYGON ((-8760347.640 4977058.004, -8759529.0..."
1,1430,17,211,COVINGTON,16656,1,033,00000,None,51.7,...,None,None,0000/00/00,0000/00/00,70707,0.0,0.0,73925.898165,2.417505e+08,"POLYGON ((-8702036.933 5020653.717, -8702068.4..."
2,273,50,218,TUSCARORA,77960,1,099,00000,None,28.3,...,None,None,0000/00/00,0000/00/00,70708,0.0,0.0,55357.033125,1.312507e+08,"POLYGON ((-8588244.220 4944468.401, -8588160.1..."
3,1303,19,501,BLOOMSBURG,07128,3,037,07128,BLMSBURG/BERWICK,4.9,...,None,None,0000/00/00,0000/00/00,70709,0.0,0.0,22388.854877,2.148478e+07,"POLYGON ((-8510415.683 5015890.684, -8510175.5..."
4,1304,20,201,ATHENS,03408,1,039,00000,None,28.5,...,None,None,0000/00/00,0000/00/00,70710,0.0,0.0,49062.562819,1.318192e+08,"POLYGON ((-8886445.911 5127362.400, -8884308.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,464,44,204,DECATUR,18464,1,087,00000,None,47.4,...,None,None,0000/00/00,0000/00/00,69774,0.0,0.0,69540.189584,2.036716e+08,"POLYGON ((-8628675.185 4966786.153, -8628500.0..."
2567,1461,15,406,KENNETT SQUARE,39352,4,029,00000,None,1.1,...,None,None,0000/00/00,0000/00/00,69775,0.0,0.0,8693.780978,4.696883e+06,"POLYGON ((-8428826.170 4844271.113, -8428811.8..."
2568,2549,66,413,HANOVER,32448,3,133,32448,HANOVER,3.0,...,None,None,0000/00/00,0000/00/00,69776,0.0,0.0,20604.772470,1.625352e+07,"POLYGON ((-8571551.215 4836188.488, -8571557.0..."
2569,1553,14,403,HOWARD,35960,1,027,00000,None,0.4,...,None,None,0000/00/00,0000/00/00,69777,0.0,0.0,5212.383423,1.666389e+06,"POLYGON ((-8644793.608 5013323.093, -8645548.4..."


In [61]:
pa_municipalities

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477645345,10690204,+41.9158651,-096.7885168,"POLYGON ((-96.55515 41.91587, -96.55515 41.914..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680976231,61568965,+46.2946377,-123.4244583,"POLYGON ((-123.49077 46.38358, -123.48813 46.3..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016818946,29090018,+34.3592729,-104.3686961,"POLYGON ((-104.38368 34.69213, -104.37658 34.6..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169272970,22847034,+40.7835474,-096.6886584,"POLYGON ((-96.68140 41.04566, -96.68139 41.045..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645185,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.04802 40.35066, -98.04674 40.350..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,13,123,00351260,13123,Gilmer,Gilmer County,06,H1,G4020,None,None,None,A,1103886633,12283371,+34.6905055,-084.4546244,"POLYGON ((-84.30237 34.57832, -84.30329 34.577..."
3230,27,135,00659513,27135,Roseau,Roseau County,06,H1,G4020,None,None,None,A,4329730562,16913419,+48.7610683,-095.8215042,"POLYGON ((-95.25857 48.88666, -95.25707 48.885..."
3231,28,089,00695768,28089,Madison,Madison County,06,H1,G4020,298,27140,None,A,1850178950,71697237,+32.6343703,-090.0341603,"POLYGON ((-90.14883 32.40026, -90.14890 32.400..."
3232,48,227,01383899,48227,Howard,Howard County,06,H1,G4020,None,13700,None,A,2333039143,8841781,+32.3034298,-101.4387208,"POLYGON ((-101.18138 32.21252, -101.18138 32.2..."


In [150]:
def add_area_and_label_coords(some_gdf):
    some_gdf['area'] = some_gdf.area
    some_gdf['coords'] = some_gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
    some_gdf['coords'] = [coords[0] for coords in some_gdf['coords']]
    
def plot_with_labels(some_gdf, ax, name_col=None, field_col=None, **kwargs):
    if field_col is None:
        some_gdf.plot(ax=ax, **kwargs)
    else:
        some_gdf.plot(column=field_col, ax=ax, legend=True, **kwargs)
    if name_col is not None:
        for idx, row in some_gdf.iterrows():
            ax.annotate(text=row[name_col], xy=row['coords'],
                         horizontalalignment='center')

def calc_overall_centroid(some_gdf):
    """
    Use the 'coords' and 'area' columns to estimate an overall centroid
    """
    sum_x = 0.0
    sum_y = 0.0
    sum_area = 0.0
    for idx, row in some_gdf.iterrows():
        coord_x, coord_y = row['coords']
        sum_x += coord_x * row['area']
        sum_y += coord_y * row['area']
        sum_area += row['area']
    centroid_x = sum_x / sum_area
    centroid_y = sum_y / sum_area
    return centroid_x, centroid_y

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees).  All args
    must be of equal length.
    """
    #print(lon1, lat1, lon2, lat2)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    #print(dlon, dlat)
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [44]:
pa_geom = pa_municipalities[pa_municipalities['STATEFP']=='42']
pa_geom

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
19,42,007,01214112,42007,Beaver,Beaver County,06,H1,G4020,430,38300,None,A,1125854819,24162295,+40.6841401,-080.3507209,"POLYGON ((-80.27803 40.53674, -80.29151 40.527..."
47,42,053,01213669,42053,Forest,Forest County,06,H1,G4020,None,None,None,A,1106595595,8369675,+41.5133040,-079.2497050,"POLYGON ((-78.95860 41.52549, -78.95861 41.525..."
95,42,117,01209189,42117,Tioga,Tioga County,06,H1,G4020,None,None,None,A,2936765818,8279717,+41.7668593,-077.2572881,"POLYGON ((-77.21159 41.54545, -77.21177 41.545..."
108,42,043,01213667,42043,Dauphin,Dauphin County,06,H1,G4020,276,25420,None,A,1359407363,86209037,+40.4125646,-076.7926343,"POLYGON ((-76.59217 40.25428, -76.59214 40.254..."
165,42,127,01213692,42127,Wayne,Wayne County,06,H1,G4020,None,None,None,A,1879734030,64785676,+41.6466021,-075.2924932,"POLYGON ((-75.29834 41.36740, -75.29849 41.367..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,42,039,01213666,42039,Crawford,Crawford County,06,H1,G4020,240,32740,None,A,2621988043,65389924,+41.6878760,-080.1077950,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."
3054,42,121,01213690,42121,Venango,Venango County,06,H1,G4020,None,36340,None,A,1746337768,22149577,+41.4007143,-079.7658126,"POLYGON ((-79.82262 41.17309, -79.82448 41.173..."
3071,42,071,01209181,42071,Lancaster,Lancaster County,06,H1,G4020,None,29540,None,A,2444751632,103368819,+40.0419921,-076.2501980,"POLYGON ((-75.97241 39.98751, -75.97422 39.984..."
3166,42,005,01213658,42005,Armstrong,Armstrong County,06,H1,G4020,430,38300,None,A,1692093647,27586728,+40.8123805,-079.4641285,"POLYGON ((-79.55165 40.60189, -79.55157 40.601..."


In [45]:
add_area_and_label_coords(pa_geom)
centroid_x, centroid_y = calc_overall_centroid(pa_geom)
print(centroid_x, centroid_y)

-77.84058346055701 40.900430189750736


C:\Users\ajmek\AppData\Local\Temp\ipykernel_14564\311135332.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  some_gdf['area'] = some_gdf.area
C:\Users\ajmek\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\ajmek\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [46]:
fig, axes = plt.subplots(1,2)
plot_with_labels(pa_geom, field_col='area', name_col='NAME', ax=axes[0])
plot_with_labels(pa_geom, ax=axes[1])
fig.suptitle('Original CRS')

<IPython.core.display.Javascript object>

Text(0.5, 0.98, 'Original CRS')

In [47]:
pa_geom.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [48]:
lat = centroid_y
lon = centroid_x
ortho = CRS.from_proj4("+proj=ortho +lat_0={} +lon_0={} +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs".format(lat, lon))

In [49]:
ortho_gdf = pa_geom.to_crs(ortho)
ortho_gdf.crs

<Projected CRS: +proj=ortho +lat_0=40.900430189750736 +lon_0=-77.8 ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Orthographic
Datum: Unknown based on Normal Sphere (r=6370997) ellipsoid
- Ellipsoid: Normal Sphere (r=6370997)
- Prime Meridian: Greenwich

In [50]:
add_area_and_label_coords(ortho_gdf)
fig, axes = plt.subplots(1,2)
plot_with_labels(ortho_gdf, field_col='area', name_col='NAME', ax=axes[0])
plot_with_labels(ortho_gdf, ax=axes[1])
fig.suptitle('Ortho')

<IPython.core.display.Javascript object>

Text(0.5, 0.98, 'Ortho')

In [51]:
merged = pd.merge(pa_snow, pa_cities, left_on='Location', right_on='Name', how='left')

In [52]:
merged

,Location,Low End Snowfall,Expected Snowfall,High End Snowfall,"P>=0.1""","P>=1""","P>=2""","P>=4""","P>=6""","P>=8""","P>=12""","P>=18""",Name,Type,County,Class,Population (2018 Estimates),Incorporation date (as city),Sq Miles
0,Bethel Park,8,12,15,100%,100%,100%,98%,94%,83%,40%,1%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Franklin Park,8,13,16,100%,100%,100%,99%,96%,88%,51%,3%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jefferson Hills,7,11,14,100%,100%,100%,98%,93%,80%,31%,1%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Monroeville,7,10,14,100%,100%,100%,99%,94%,81%,32%,1%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Moon/Pittsburgh Intl Airport,9,13,16,100%,100%,100%,99%,97%,91%,56%,4%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Ligonier,7,9,12,100%,100%,100%,100%,94%,71%,14%,0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,Lower Burrell,7,10,14,100%,100%,100%,99%,95%,82%,32%,1%,Lower Burrell,City,Westmoreland,Third,11228.0,1959.0,11.26
90,Monessen,7,10,13,100%,100%,100%,99%,93%,76%,26%,1%,Monessen,City,Westmoreland,Third,7339.0,1921.0,2.89
91,Murrysville,7,10,14,100%,100%,100%,99%,96%,82%,31%,1%,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
geolocator = Nominatim(user_agent="notme")
def get_county(location):
    full_location = f"{location}, Pennsylvania, US"
    geo_data = geolocator.geocode(full_location)
    if geo_data:
        address = geo_data.address
        parts = address.split(", ")
        for part in parts:
            if "County" in part:
                return part.split()[0]
    return None

merged["County"] = merged.apply(lambda row: get_county(row["Location"]) if pd.isna(row["County"]) else row["County"], axis=1)

merged = merged.dropna(subset=["County"])

merged_grouped = merged.groupby("County", as_index=False)["Expected Snowfall"].mean()

In [54]:
merged_grouped

,County,Expected Snowfall
0,Allegheny,11.571429
1,Armstrong,9.500000
2,Beaver,12.400000
3,Butler,12.200000
4,Clarion,11.500000
5,Clearfield,10.000000
6,Fayette,9.666667
7,Forest,12.000000
8,Greene,9.800000
9,Huntingdon,9.000000


In [55]:
geo_merged = pd.merge(merged_grouped, pa_municipalities, left_on='County', right_on='NAME', how='outer')
geo_merged = gpd.GeoDataFrame(geo_merged, geometry="geometry")

In [56]:
geo_merged

,County,Expected Snowfall,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,NaN,NaN,45,001,01245666,45001,Abbeville,Abbeville County,06,H1,G4020,None,None,None,A,1272184102,53221051,+34.2290410,-082.4540577,"POLYGON ((-82.50085 34.07190, -82.50093 34.071..."
1,NaN,NaN,22,001,00558389,22001,Acadia,Acadia Parish,15,H1,G4020,318,29180,None,A,1697073813,5934788,+30.2914970,-092.4110366,"POLYGON ((-92.36150 30.07855, -92.36294 30.077..."
2,NaN,NaN,51,001,01480091,51001,Accomack,Accomack County,06,H1,G4020,None,None,None,A,1163725929,2229259960,+37.7659435,-075.7578073,"POLYGON ((-75.51693 37.65483, -75.51720 37.653..."
3,NaN,NaN,16,001,00395066,16001,Ada,Ada County,06,H1,G4020,147,14260,None,A,2724705225,22159206,+43.4514767,-116.2443760,"POLYGON ((-115.98028 43.58595, -115.98018 43.5..."
4,NaN,NaN,40,001,01101788,40001,Adair,Adair County,06,H1,G4020,None,None,None,A,1486337778,8872711,+35.8980428,-094.6510258,"POLYGON ((-94.79743 35.96173, -94.79743 35.961..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,NaN,NaN,04,027,00023901,04027,Yuma,Yuma County,06,H1,G4020,None,49740,None,A,14280774791,13253159,+32.7739424,-113.9109050,"POLYGON ((-114.47325 33.02788, -114.45989 33.0..."
3230,NaN,NaN,08,125,00198178,08125,Yuma,Yuma County,06,H1,G4020,None,None,None,A,6123764029,11134194,+40.0007703,-102.4216491,"POLYGON ((-102.05173 39.97827, -102.05173 39.9..."
3231,NaN,NaN,48,505,01384038,48505,Zapata,Zapata County,06,H1,G4020,None,49820,None,A,2585876042,154371160,+26.9969806,-099.1826027,"POLYGON ((-99.15217 27.27142, -99.15088 27.271..."
3232,NaN,NaN,48,507,01384039,48507,Zavala,Zavala County,06,H1,G4020,None,None,None,A,3360268878,11208707,+28.8646523,-099.7598304,"POLYGON ((-99.41160 28.81923, -99.41159 28.818..."


In [57]:
pa_geom = geo_merged[geo_merged['STATEFP']=='42']
pa_geom.plot(column="Expected Snowfall", legend=True, missing_kwds={'color': 'lightgrey', 'label': 'Missing Values', 'hatch':'///', 'edgecolor': 'lightgrey'})


<IPython.core.display.Javascript object>

<Axes: >

In [58]:
travel_towns = ['Pittsburgh',
                'Scranton',
        'Allentown',
        'Philadelphia',
        'Harrisburg',
        'Pittsburgh']
pa_municipalities.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [66]:
gdf = gpd.read_file('PaMunicipalities2022_01.shp', engine='pyogrio')
gdf

,MSLINK,COUNTY,MUNICIPAL_,MUNICIPAL1,FIPS_MUN_C,FED_AID_UR,FIPS_COUNT,FIPS_AREA_,FIPS_NAME,FIPS_SQ_MI,...,FED_ID_NUM,CLASS_OF_M,GIS_UPDATE,GIS_GEOMET,GPID,GEOMETRY_A,GEOMETRY_L,Shape_Leng,Shape_Area,geometry
0,2254,59,209,WEST BUFFALO,82640,1,119,00000,None,38.2,...,24-6001593,2TWP,0000/00/00,0000/00/00,33785,0.0,0.0,74680.772799,1.773021e+08,"MULTIPOLYGON (((-8574663.772 5016595.345, -857..."
1,1258,21,203,HOPEWELL,35672,1,041,00000,None,28.2,...,23-2131817,2TWP,0000/00/00,0000/00/00,33786,0.0,0.0,55735.307406,1.241206e+08,"POLYGON ((-8625458.082 4884023.359, -8625540.0..."
2,2255,59,210,WHITE DEER,84496,2,119,50016,MILTON,30.6,...,24-6001596,2TWP,0000/00/00,0000/00/00,33787,0.0,0.0,63730.550468,2.107778e+08,"POLYGON ((-8556199.065 5016219.612, -8556221.3..."
3,283,50,407,MILLERSTOWN,49720,1,099,00000,None,1.2,...,23-6050573,BORO,0000/00/00,0000/00/00,33788,0.0,0.0,10891.156452,4.335253e+06,"POLYGON ((-8587283.283 4946353.019, -8587324.0..."
4,2278,60,219,SCRUBGRASS,69016,1,121,00000,None,27.4,...,25-1377305,2TWP,0000/00/00,0000/00/00,33789,0.0,0.0,73267.778096,1.259692e+08,"POLYGON ((-8873654.185 5038436.781, -8873662.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,1926,05,409,RAINSBURG,63312,1,009,00000,None,0.2,...,23-7345258,BORO,0000/00/00,0000/00/00,35286,0.0,0.0,4660.742508,7.157302e+05,"POLYGON ((-8740491.999 4851394.613, -8740375.4..."
2568,930,32,209,CHERRYHILL,13072,1,063,00000,None,48.4,...,25-6000790,2TWP,0000/00/00,0000/00/00,35287,0.0,0.0,78591.520019,2.210764e+08,"POLYGON ((-8800348.188 4960884.886, -8796315.5..."
2569,1905,06,231,PIKE,60176,1,011,00000,None,14.3,...,23-6000225,2TWP,0000/00/00,0000/00/00,35288,0.0,0.0,34408.089886,6.235252e+07,"POLYGON ((-8427369.189 4929344.422, -8421718.1..."
2570,1992,04,419,INDUSTRY,36944,4,007,00000,None,9.4,...,25-1210216,BORO,0000/00/00,0000/00/00,35289,0.0,0.0,44971.557579,4.924139e+07,"POLYGON ((-8953845.635 4965678.108, -8952045.5..."


In [141]:
base = gdf
pa_platte = base.to_crs(epsg=4326)
pa_ortho = base.to_crs(epsg=32618)
travel_towns = [x.upper() for x in travel_towns]

pa_travel_platte = pa_platte[pa_platte["MUNICIPAL1"].isin(travel_towns)]
pa_travel_ortho = pa_ortho[pa_ortho["MUNICIPAL1"].isin(travel_towns)]
pa_travel_platte = pa_travel_platte.set_index("MUNICIPAL1").loc[travel_towns].reset_index()
pa_travel_ortho = pa_travel_ortho.set_index("MUNICIPAL1").loc[travel_towns].reset_index()

pa_travel_ortho.iloc[-1] = pa_travel_ortho.iloc[0]
pa_travel_platte.iloc[-1] = pa_travel_platte.iloc[0]

add_area_and_label_coords(pa_travel_platte)
add_area_and_label_coords(pa_travel_ortho)
pa_travel_ortho

C:\Users\ajmek\AppData\Local\Temp\ipykernel_14564\311135332.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  some_gdf['area'] = some_gdf.area


,MUNICIPAL1,MSLINK,COUNTY,MUNICIPAL_,FIPS_MUN_C,FED_AID_UR,FIPS_COUNT,FIPS_AREA_,FIPS_NAME,FIPS_SQ_MI,...,GIS_UPDATE,GIS_GEOMET,GPID,GEOMETRY_A,GEOMETRY_L,Shape_Leng,Shape_Area,geometry,area,coords
0,PITTSBURGH,2140,02,301,61000,4,003,00009,PITTSBURGH,58.2,...,0000/00/00,0000/00/00,34021,0.0,0.0,161680.100738,2.609306e+08,"POLYGON ((69897.705 4491327.326, 69927.898 449...",1.515256e+08,"(78734.67607713833, 4487429.74811196)"
1,SCRANTON,852,35,302,69000,4,069,00281,SCRAN/WLKS-BARRE,26.0,...,0000/00/00,0000/00/00,33766,0.0,0.0,70454.442512,1.172628e+08,"POLYGON ((442122.809 4580270.925, 441909.285 4...",6.586511e+07,"(444803.14922898845, 4583864.67206875)"
2,ALLENTOWN,703,39,301,02000,4,077,00068,LEHIGH VALLEY,18.3,...,0000/00/00,0000/00/00,35178,0.0,0.0,81221.859145,8.098603e+07,"POLYGON ((457698.200 4495965.770, 457817.185 4...",4.660955e+07,"(459375.7494195444, 4493715.990069248)"
3,PHILADELPHIA,2573,67,301,60000,4,101,00004,PHILADELPHIA,143.0,...,0000/00/00,0000/00/00,34751,0.0,0.0,157206.179534,6.297380e+08,"POLYGON ((499057.954 4430168.212, 499036.214 4...",3.687349e+08,"(488092.7592549054, 4428043.571485484)"
4,HARRISBURG,1212,22,301,32800,4,043,00083,HARRISBURG,11.6,...,0000/00/00,0000/00/00,33456,0.0,0.0,44248.628005,5.200551e+07,"POLYGON ((343558.460 4458078.484, 343533.663 4...",3.023312e+07,"(338729.55430151377, 4460773.425978204)"
5,PITTSBURGH,2140,02,301,61000,4,003,00009,PITTSBURGH,58.2,...,0000/00/00,0000/00/00,34021,0.0,0.0,161680.100738,2.609306e+08,"POLYGON ((69897.705 4491327.326, 69927.898 449...",1.515256e+08,"(78734.67607713833, 4487429.74811196)"


In [142]:
fig, ax = plt.subplots(figsize=(8, 6))
#pa_ortho.plot(ax=ax, color="lightgrey", edgecolor="gray", label="Pennsylvania")
#pa_travel_ortho.plot(ax=ax, color="red", markersize=100, label="Travel Cities")
pa_geom.plot(ax=ax, color='none', edgecolor='gray')
plot_with_labels(pa_travel_platte, name_col='MUNICIPAL1', ax=ax)
for i in range(1, len(pa_travel_platte)):
    start_coords = pa_travel_platte['coords'].iloc[i - 1]
    end_coords = pa_travel_platte['coords'].iloc[i]
    ax.annotate('', xy=end_coords, xytext=start_coords, arrowprops=dict(facecolor='blue', edgecolor='blue', arrowstyle='->'))
plt.show()


<IPython.core.display.Javascript object>

In [156]:
pa_travel_ortho['centroid'] = pa_travel_ortho.geometry.centroid
ortho_dist = 0
for i in range(1, len(pa_travel_ortho)):
    start_centroid = pa_travel_ortho['centroid'].iloc[i - 1]
    end_centroid = pa_travel_ortho['centroid'].iloc[i]
    ortho_dist += start_centroid.distance(end_centroid)
start_centroid = pa_travel_ortho['centroid'].iloc[-1]
end_centroid = pa_travel_ortho['centroid'].iloc[0]
ortho_dist += start_centroid.distance(end_centroid)

platte_dist = 0
for i in range(1, len(pa_travel_platte)):
    start_coords = pa_travel_platte['coords'].iloc[i - 1]
    end_coords = pa_travel_platte['coords'].iloc[i]
    lon1, lat1 = start_coords
    #print(lon1, lat1)
    lon2, lat2 = end_coords
    #print(lon2, lat2)
    #temp = haversine_np(lon1, lat1, lon2, lat2)
    platte_dist += haversine_np(lon1, lat1, lon2, lat2)

print(f"Ortho Distance Between Cities: {ortho_dist} meters, or {ortho_dist / 1000} km.")
print(f"Platte Distance Between Cities: {platte_dist * 1000} meters, or {platte_dist} km.")

# One explanation as to why the two distance values are different is that each projection alters its perception of the world differently to match a certain trait or desired vision one projection has over another, so travel distance slightly varies between the two projections.

Ortho Distance Between Cities: 956866.8613047686 meters, or 956.8668613047686 km.
Platte Distance Between Cities: 955384.8741601749 meters, or 955.3848741601748 km.


'\nOne explanation as to why the two distance values\n'